<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/TreeEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘으론 앙상블 학습이 있다.
# 비정형 데이터는 규칙성을 찾기 어려워서 머신러닝 알고리즘보단 신경막 알고리즘을 써야 한다.

In [ ]:
# 랜덤 포레스트 기법은 앙상블 학습의 대표 주자다.
# 과정 : 결정 트리 랜덤하게 만들기 -> 숲 만들기 -> 각 결정 트리의 예측을 사용해 최종 예측 만들기
# 랜덤 포레스트에서는 훈련용 데이터를 부트스트랩 샘플 방식으로 추출한다. (중복 허용해서 데이터 샘플링)

In [ ]:
# 랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하므로 훈련세트에 과대 적합되는 것을 막아주고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine[['class']].to_numpy()

train_input,test_input,train_target,test_target=train_test_split(
    data,target,test_size=0.2,random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
# n_jobs = -1 을 해서 최대한 병렬로 교차 검증
rf=RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

In [ ]:
# 부트스트랩 샘플에 포함되지 않은 샘플인 OOB 샘플을 활용하여, 부트스트랩 샘플로 훈련한 결정 트리를 평가 가능
# 즉, OOB 샘플로 검증 세트의 역할을 할 수 있음.
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

In [ ]:
# 엑스트라 트리는 부트스트랩 샘플을 활용하여 뽑지 않음(결정 트리 만들 때 전체 세트 사용). 대신 노드를 분할할 떄 가장 좋은 분할을 하는 것이 아니라, 무작위로 분할함
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
et.fit(train_input,train_target)
print(et.feature_importances_)

In [ ]:
# 그레디언트 부스팅은 깊이가 얕은 결정 트리를 사용해서 이진 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
# 경사 하강법을 사용하여 트리를 앙상블에 추가.
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)

scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
# 학습률 증가, 트리의 개수 늘리기
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)

scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

In [ ]:
# 히스토그램 기반 그래디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
from sklearn.inspection import permutation_importance # 특성 중요도 계산

hgb.fit(train_input,train_target)
result=permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1) # 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰, 어떤 특성이 중요한 지 계산한다.
print(result.importances_mean)

In [ ]:
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

In [ ]:
hgb.score(test_input,test_target)

In [ ]:
from xgboost import XGBClassifier

xgb=XGBClassifier(tree_method='hist',random_state=42)
scores=cross_validate(xgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

In [ ]:
from lightgbm import LGBMClassifier

lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))